# This notebook shows an example where a set of electrodes are selected from a dataset and then LFP is extracted from those electrodes and then written to a new NWB file


## The following steps must be completed before running this notebook:
### 1) Set up access to a DataJoint database. See https://tutorials.datajoint.io/
### 2) Install pynwb. 
    See https://pynwb.readthedocs.io/en/stable/getting_started.html#installation
    Note: currently this requires pynwb 1.3.3 or above and hdmf 2.0.1 or above, so use the latest development versions, not the conda versions
### 3) Install the ndx-fl-novela package. 
     conda install -c novelakrk ndx-fl-novela
### 4) Download the example franklab NWB file from DropBox. 
    https://www.dropbox.com/to_be_updated

#### Make sure we're in the franklabnwb/franklabnwb directory. 
#### Eventually this will be unnecessary because the package will properly installed

#### Load all of the relevant modules and set the environment variables. 
Note that the datadir and datadir/analysis must exist

In [2]:
%env DJ_SUPPORT_FILEPATH_MANAGEMENT=TRUE
%load_ext autoreload
%autoreload 2


import pynwb
import os

#DataJoint and DataJoint schema
import datajoint as dj

# the commands below can be run once to update your global configuration

dj.config["enable_python_native_blobs"] = True
dj.config["database.user"] = 'root'
dj.config["database.password"] = 'tutorial'
dj.config.save_global()

import nwb_datajoint as nd
import ndx_franklab_novela.probe

data_dir = '/Users/loren/data/nwb_builder_test_data'
os.environ['NWB_DATAJOINT_BASE_DIR'] = data_dir
os.environ['KACHERY_STORAGE_DIR'] = os.path.join(data_dir, 'kachery-storage')



env: DJ_SUPPORT_FILEPATH_MANAGEMENT=TRUE
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Next we select the NWB file, which corresponds to the dataset we want to extract LFP from

In [3]:
nwb_file_names = nd.common.Nwbfile().fetch('nwb_file_name')
# take the first one for this demonstration
nwb_file_name = nwb_file_names[0]
print(nwb_file_name)

beans20190718.nwb


#### Create the standard LFP Filters. This only needs to be done once.

In [4]:
nd.common.FirFilter().create_standard_filters()

### Select every fourth electrode for LFP

In [6]:
electrode_ids = nd.common.Electrode.fetch('electrode_id')
lfp_electrode_ids = electrode_ids[range(0, len(electrode_ids), 4)]
nd.common.LFPSelection().set_lfp_electrodes(nwb_file_name, lfp_electrode_ids.tolist())


About to delete:
Nothing to delete


In [8]:
nd.common.LFPSelection().LFPElectrode()

nwb_file_name the name of the NWB file,electrode_group_name electrode group name from NWBFile,electrode_id the unique number for this electrode
beans20190718.nwb,0,0
beans20190718.nwb,0,4
beans20190718.nwb,0,8
beans20190718.nwb,0,12
beans20190718.nwb,0,16
beans20190718.nwb,0,20
beans20190718.nwb,0,24
beans20190718.nwb,0,28
beans20190718.nwb,0,32
beans20190718.nwb,0,36


### Or select one electrode for LFP


In [6]:
nd.common.LFPSelection().set_lfp_electrodes(nwb_file_name, [2])

About to delete:
Nothing to delete


### Populate the LFP table. Note that this takes 2 hours or so on a laptop if you use all electrodes

In [ ]:
nd.common.LFP().populate()

writing new NWB file beans20190718_00000001.nwb
Output array should have shape (1091950, 64) and dtype <f8
Output array should have shape (212812, 64) and dtype <f8
Output array should have shape (2557744, 64) and dtype <f8
Output array should have shape (1356939, 64) and dtype <f8
Output array should have shape (1680945, 64) and dtype <f8
Checking output array shape is disabled, make sure portion of output array has shape (1091950, 64)
Checking output array shape is disabled, make sure portion of output array has shape (212812, 64)


### Now that we've created the LFP object we can perform a second level of filtering for a band of interest, in this case the theta band
We first need to create the filter

In [ ]:
lfp_sampling_rate = (nd.common.LFP() & {'nwb_file_name' : nwb_file_name}).fetch1('lfp_sampling_rate')
filter_name = 'Theta 5-11 Hz'
nd.common.FirFilter().add_filter(filter_name, lfp_sampling_rate, 'bandpass', [4, 5, 11, 12], 'theta filter for 1 Khz data')

Next we add an entry for the LFP Band and the electrodes we want to filter

In [ ]:
# assume that we've filtered all the electrodes; change this if not
lfp_band_electrode_ids = [0, 4, 8]

# set the interval list name corresponding to the second epoch (a run session)
interval_list_name = '02_r1'

# set the reference to -1 to indicate no reference for all channels
ref_elect = [-1]

# desired sampling rate
lfp_band_sampling_rate = lfp_sampling_rate // 10

In [ ]:
nd.common.LFPBandSelection().set_lfp_band_electrodes(nwb_file_name, lfp_band_electrode_ids, filter_name, interval_list_name, ref_elect, lfp_band_sampling_rate)

Check to make sure it worked

In [ ]:
nd.common.LFPBandSelection().LFPBandElectrode()

In [ ]:
(nd.common.LFP() & {'nwb_file_name' : nwb_file_name}).fetch_nwb(limit=1)[0]['lfp']

In [ ]:
nd.common.LFPBand().populate()

### Now we can plot the original signal, the LFP filtered trace, and the theta filtered trace together.
Much of the code below could be replaced by a function calls that would return the data from each electrical series

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#get the three electrical series objects and the indeces of the electrodes we band pass filtered
orig_eseries = (nd.common.Raw() & {'nwb_file_name' : nwb_file_name}).fetch_nwb()[0]['nwb']
orig_elect_indeces = nd.common.get_electrode_indeces(orig_eseries, lfp_band_electrode_ids)

lfp_eseries = (nd.common.LFP() & {'nwb_file_name' : nwb_file_name}).fetch_nwb()[0]['lfp']
lfp_elect_indeces = nd.common.get_electrode_indeces(lfp_eseries, lfp_band_electrode_ids)

lfp_band_eseries = (nd.common.LFPBand() & {'nwb_file_name' : nwb_file_name}).fetch_nwb()[0]['filtered_data']
lfp_band_elect_indeces = nd.common.get_electrode_indeces(lfp_band_eseries, lfp_band_electrode_ids)


In [ ]:
# get a list of times for the first run epoch and then select a 2 second interval 100 seconds from the beginning
run1times = (nd.common.IntervalList & {'interval_list_name' : '02_r1'}).fetch1('valid_times')
plottimes = [run1times[0][0] + 101, run1times[0][0] + 102]

In [ ]:
# get the time indeces for each dataset
orig_time_ind = np.argwhere(np.logical_and(orig_eseries.timestamps > plottimes[0], orig_eseries.timestamps < plottimes[1]))
lfp_time_ind = np.argwhere(np.logical_and(lfp_eseries.timestamps > plottimes[0], lfp_eseries.timestamps < plottimes[1]))
lfp_band_time_ind = np.argwhere(np.logical_and(lfp_band_eseries.timestamps > plottimes[0], lfp_band_eseries.timestamps < plottimes[1]))

In [ ]:
plt.plot(orig_eseries.timestamps[orig_time_ind], orig_eseries.data[orig_time_ind,orig_elect_indeces[1]], 'k-')
plt.plot(lfp_eseries.timestamps[lfp_time_ind], lfp_eseries.data[lfp_time_ind,lfp_elect_indeces[1]], 'b-')
plt.plot(lfp_band_eseries.timestamps[lfp_band_time_ind], lfp_band_eseries.data[lfp_band_time_ind,lfp_band_elect_indeces[1]], 'r-')
plt.xlabel('Time (sec)')
plt.ylabel('Amplitude (AD units)')

plt.show()